In [1]:
%%configure -f
{
    "conf": {
        "spark.app.name": "stop_conn",
        "spark.executor.memory": "2g",
        "spark.executor.instances": "32"
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8361,application_1589299642358_2893,pyspark,idle,Link,Link,
8374,application_1589299642358_2906,pyspark,idle,Link,Link,
8376,application_1589299642358_2908,pyspark,idle,Link,Link,
8387,application_1589299642358_2919,pyspark,idle,Link,Link,
8389,application_1589299642358_2921,pyspark,idle,Link,Link,
8390,application_1589299642358_2922,pyspark,idle,Link,Link,
8391,application_1589299642358_2923,pyspark,idle,Link,Link,
8392,application_1589299642358_2924,pyspark,idle,Link,Link,
8393,application_1589299642358_2925,pyspark,idle,Link,Link,
8394,application_1589299642358_2926,pyspark,idle,Link,Link,


In [2]:
spark

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8408,application_1589299642358_2940,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
8361,application_1589299642358_2893,pyspark,idle,Link,Link,
8374,application_1589299642358_2906,pyspark,idle,Link,Link,
8376,application_1589299642358_2908,pyspark,idle,Link,Link,
8387,application_1589299642358_2919,pyspark,idle,Link,Link,
8389,application_1589299642358_2921,pyspark,idle,Link,Link,
8390,application_1589299642358_2922,pyspark,idle,Link,Link,
8391,application_1589299642358_2923,pyspark,idle,Link,Link,
8392,application_1589299642358_2924,pyspark,idle,Link,Link,
8393,application_1589299642358_2925,pyspark,idle,Link,Link,
8394,application_1589299642358_2926,pyspark,idle,Link,Link,


In [4]:
import pyspark.sql.functions as f
from pyspark.sql import Window

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
trips = spark.read.orc("/data/sbb/timetables/orc/trips/")
calendar = spark.read.orc("/data/sbb/timetables/orc/calendar/")
routes = spark.read.orc("/data/sbb/timetables/orc/routes/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
trips.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+----------+--------------------+------------------+---------------+------------+
|   route_id|service_id|             trip_id|     trip_headsign|trip_short_name|direction_id|
+-----------+----------+--------------------+------------------+---------------+------------+
|1-1-C-j19-1|  TA+b0001|5.TA.1-1-C-j19-1.3.R|Zofingen, Altachen|            108|           1|
+-----------+----------+--------------------+------------------+---------------+------------+
only showing top 1 row

In [7]:
calendar.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+------+-------+---------+--------+------+--------+------+
|service_id|monday|tuesday|wednesday|thursday|friday|saturday|sunday|
+----------+------+-------+---------+--------+------+--------+------+
|  TA+b0nx9|  true|   true|     true|    true|  true|   false| false|
+----------+------+-------+---------+--------+------+--------+------+
only showing top 1 row

In [8]:
routes.show(1)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------+---------+----------------+---------------+----------+----------+
|   route_id|agency_id|route_short_name|route_long_name|route_desc|route_type|
+-----------+---------+----------------+---------------+----------+----------+
|11-40-j19-1|      801|             040|               |       Bus|       700|
+-----------+---------+----------------+---------------+----------+----------+
only showing top 1 row

In [9]:
# select all services that are scheduled on mondays
monday_service_id = (
    calendar
    .where(calendar.monday == True)
    .select('service_id')
    .dropDuplicates()
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# keep only trips that are scheduled on mondays
monday_trips = (
    trips
    .join(f.broadcast(monday_service_id), 'service_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
# add route information to trips
route_trips = (
    trips
    .join(routes, 'route_id')
    
    # drop all transports that are not of type:
    # InterCity, InterRegio, Regionalzug, S-Bahn, Bus, Tram
    .where(routes.route_type.isin(['102', '103', '106', '400', '700', '900']))
    
    # create id that is used to merge with ist data
    .withColumn(
        'id',
        f.when(routes.route_type.isin(['700', '900']),
            f.concat_ws(':', f.lit('85'), routes.agency_id.cast('int').cast('string'), routes.route_short_name.cast('int').cast('string'))
        ).otherwise(
            f.concat_ws(':', f.lit('85'), routes.agency_id.cast('int').cast('string'), trips.trip_short_name.cast('int').cast('string'))
        )
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
route_trips.write.format('orc').save('/user/datavirus/route_trips.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
route_trips = spark.read.orc('/user/datavirus/route_trips.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [74]:
# add delay probabilities to each route
# route_probability = (
#     route_trips
#     .join(probability, 'id')
# )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [75]:
# route_probability.write.format('orc').save('/user/datavirus/route_probability.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
# route_probability = spark.read.orc('/user/datavirus/route_probability.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
# route_probability.show(5, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
# read timetable stop times and filter values we are interested in

stop_time = spark.read.orc("/data/sbb/timetables/orc/stop_times/")

stop_time = (
    stop_time
    .select(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence'])
    .filter(
        (stop_time.departure_time >= '05:00:00')
        & (stop_time.departure_time <= '21:00:00')
        & (stop_time.arrival_time >= '05:00:00')
        & (stop_time.arrival_time <= '21:00:00') 
    )
    .withColumn('station_id', stop_time.stop_id.substr(0, 7))
    .drop('stop_id')
    
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [79]:
# load stations around zurich

stations = spark.read.csv("../data/zurich_stations_ids.csv")
stations = (
    stations
    .select(stations._c0.alias('station_id'))
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [80]:
# remove all stops that are not close to zurich

stop_filtered = (
    stop_time
    .join(f.broadcast(stations), 'station_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [81]:
# because we removed some stops there are now trips with a single stop
# we only want to keep trips with multiple stops

stop_keep = (
    stop_filtered
    .groupBy('trip_id')
    .count()
    .filter('count > 1')
    .select('trip_id')
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [82]:
# because we removed stops there are now gaps in the stop_sequence
# we recreate the stop_sequence with a rank function

trip_window = Window.partitionBy('trip_id').orderBy(f.asc('stop_sequence'))
stop_sequence = f.rank().over(trip_window)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [83]:
# reset the stop_sequence

stop_resequenced = (
    stop_filtered
    .join(stop_keep, 'trip_id')
    .withColumn('stop_sequence', stop_sequence)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [84]:
stop_resequenced.write.format('orc').save('/user/datavirus/stop_resequenced.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
stop_resequenced = spark.read.orc('/user/datavirus/stop_resequenced.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
# we want to join the resequenced dataframe
# with itself but with the stop_sequence
# of the second dataframe shifted by one
# for this we first have to create the
# two dataframes

departure = spark.read.orc('/user/datavirus/stop_resequenced.orc').alias('departure')
arrival = spark.read.orc('/user/datavirus/stop_resequenced.orc').alias('arrival')

In [28]:
# join the dataframes, this gives us all connections

raw_connections = (
    departure
    .join(arrival, ((f.col('departure.trip_id') == f.col('arrival.trip_id')) & (f.col('departure.stop_sequence') == (f.col('arrival.stop_sequence') - 1))))
    .select(
        f.col('departure.stop_sequence').alias('stop_sequence'),
        f.col('departure.trip_id').alias('trip_id'),
        f.col('departure.station_id').alias('start_id'),
        f.col('departure.departure_time').alias('start_time'),
        f.col('arrival.arrival_time').alias('stop_time'),
        f.col('arrival.station_id').alias('stop_id')
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
raw_connections.write.format('orc').save('/user/datavirus/raw_connections_new.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

KeyboardInterrupt: 

In [ ]:
raw_connections = spark.read.orc('/user/datavirus/raw_connections_new.orc')

In [ ]:
raw_connections.show(10, False)

In [13]:
id_connections = (
    raw_connections
    .join(route_trips, 'trip_id')
    .withColumn('arrival_time_minute', raw_connections.stop_time.substr(0, 5))
    .withColumn('arrival_time_hour', raw_connections.stop_time.substr(0, 2))
    .withColumn('station_id', raw_connections.stop_id)
    .withColumn(
        'produkt_id', 
        f.when(
            route_trips.route_type == '700',
            f.lit('bus')            
        ).otherwise(
            f.when(
                route_trips.route_type == '900',
                f.lit('tram')
            ).otherwise(f.lit('zug'))
        )
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
id_connections.write.format('orc').save('/user/datavirus/id_connections_new.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
id_connections = spark.read.orc('/user/datavirus/id_connections_new.orc').repartition(150, 'id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
id_connections.count()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

1663611

In [17]:
probability = spark.read.orc('/user/datavirus/probability.orc').repartition(150, 'id')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
# we add the probability information to all connections

probability_connections = (
    id_connections
    .join(probability, ['id', 'station_id', 'arrival_time_minute'], how='left_outer')
    .select(
        id_connections.stop_sequence,
        id_connections.route_type,
        id_connections.arrival_time_hour,
        id_connections.produkt_id,
        id_connections.start_id,
        id_connections.start_time,
        id_connections.trip_id,
        probability.transport_type,
        probability.line_text,
        id_connections.stop_time,
        id_connections.stop_id,
        probability.delay_probability,
        probability.delay_parameter
    )
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
probability_connections.write.format('orc').save('/user/datavirus/probability_connections_new.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
probability_connections = spark.read.orc('/user/datavirus/probability_connections_new.orc').alias('probability_connections')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
# for some connections we have no line specific probabilities 
# so we have to add transport probabilities as backup

transport_probability = spark.read.orc('/user/datavirus/transport_probability.orc').alias('transport_probability')
transport_probability = (
    transport_probability
    .withColumn('arrival_time_hour', transport_probability.ankunftszeit_hour)
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
# we get the final dataframe

connections = (
    probability_connections
    .join(transport_probability, ['produkt_id', 'arrival_time_hour'])
    .select(
        probability_connections.stop_sequence,
        probability_connections.route_type,
        probability_connections.start_id,
        probability_connections.start_time,
        probability_connections.trip_id,
        probability_connections.produkt_id.alias('transport_type'),
        probability_connections.line_text,
        probability_connections.stop_time,
        probability_connections.stop_id,
        f.when(
            f.col('probability_connections.delay_probability').isNotNull(),
            f.col('probability_connections.delay_probability')
        ).otherwise(
            f.col('transport_probability.transport_delay_probability')
        ).alias('delay_probability'),
        f.when(f.col('probability_connections.delay_parameter').isNotNull(),
              f.col('probability_connections.delay_parameter')
        ).otherwise(
            f.col('transport_probability.transport_delay_parameter')
        ).alias('delay_parameter')
    )
    .orderBy([probability_connections.stop_time.desc(), probability_connections.stop_sequence.desc()])
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
connections.write.format('orc').save('/user/datavirus/connections_new.orc', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
connections = spark.read.orc('/user/datavirus/connections_new.orc')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
connections.write.format('csv').save('/user/datavirus/connections.csv', mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
connections = spark.read.csv('/user/datavirus/connections.csv')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
connections.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---+---+-------+--------+-----------------------+---+----+--------+-------+------------------+--------------------+
|_c0|_c1|_c2    |_c3     |_c4                    |_c5|_c6 |_c7     |_c8    |_c9               |_c10                |
+---+---+-------+--------+-----------------------+---+----+--------+-------+------------------+--------------------+
|2  |700|8502209|10:05:00|9.TA.30-170-Y-j19-1.1.H|bus|null|10:05:00|8502209|0.9193866847990115|0.011589134124321832|
|2  |700|8502771|10:05:00|392.TA.26-235-j19-1.5.R|bus|235 |10:05:00|8502771|0.9159136766383016|0.013732196842403623|
|2  |700|8502771|10:05:00|493.TA.26-235-j19-1.5.R|bus|235 |10:05:00|8502771|0.9159136766383016|0.013732196842403623|
|2  |700|8502771|10:05:00|360.TA.26-235-j19-1.5.R|bus|235 |10:05:00|8502771|0.9159136766383016|0.013732196842403623|
|2  |700|8502771|10:05:00|345.TA.26-235-j19-1.5.R|bus|235 |10:05:00|8502771|0.9159136766383016|0.013732196842403623|
|2  |700|8502771|10:05:00|439.TA.26-235-j19-1.5.R|bus|235 |10:05

In [13]:
connections.write.csv('/user/datavirus/connections_test.csv', header=True, mode='overwrite')

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
c = spark.read.csv('/user/datavirus/connections_test.csv', header=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
c.show(10, False)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+----------+--------+----------+-----------------------+--------------+---------+---------+-------+------------------+--------------------+
|stop_sequence|route_type|start_id|start_time|trip_id                |transport_type|line_text|stop_time|stop_id|delay_probability |delay_parameter     |
+-------------+----------+--------+----------+-----------------------+--------------+---------+---------+-------+------------------+--------------------+
|2            |700       |8502209 |10:05:00  |9.TA.30-170-Y-j19-1.1.H|bus           |null     |10:05:00 |8502209|0.9193866847990115|0.011589134124321832|
|2            |700       |8502771 |10:05:00  |392.TA.26-235-j19-1.5.R|bus           |235      |10:05:00 |8502771|0.9159136766383016|0.013732196842403623|
|2            |700       |8502771 |10:05:00  |493.TA.26-235-j19-1.5.R|bus           |235      |10:05:00 |8502771|0.9159136766383016|0.013732196842403623|
|2            |700       |8502771 |10:05:00  |360.TA.26-235-j19-1.5.R|bus   